**Fase 4: Extracción de Tablas de los Premios Oscar con Beautiful Soup**

En esta fase, trabajarán con la biblioteca Beautiful Soup para extraer información relevante de las tablas de los premios Oscar desde 1990. La tabla la encontrarás en el este link. La información que deberás sacar es:

 - Fecha de la ceremonia.
 - Mejor película.
 - Mejor director.
 - Mejor actor.
 - Mejor actriz.

NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. Siguiendo el siguiente ejemplo:

  *[(1990, 'Driving Miss Daisy', 'O. Stone', "D. Day-Lewis",' J. Tandy' ),  ...]*

In [ ]:
from bs4 import BeautifulSoup

import requests
import pandas as pd
import os

import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np


In [ ]:
#[(1990, 'Driving Miss Daisy', 'O. Stone', "D. Day-Lewis",' J. Tandy' ),  ...]


url_oscar = "https://es.wikipedia.org/wiki/Premios_%C3%93scar"
res_oscar = requests.get(url_oscar)
sopa_oscar = BeautifulSoup(res_oscar.content, "html.parser") #traigo html
tabla_oscar = sopa_oscar.find("table", {"class": "wikitable"}) 
#en las tablas de wikipedia suelen tener "wikitable" como clase

# una vez tenemos la tabla:
# 1.sacamos todas las filas
# 2. una vez tenemos todas las filas, identifico la columna
# 3. Una vez tengo cada columnas, sacamos lo que nos interesa,nos piden el año solo(parseamos solo el año)
# una vez tenemos todos los valores, dejamos solo las filas y columnas que nos interesan:
#   fecha(año ceremonia),mejor peli, mejor director, mejor actor, mejor actriz
#   1990-2023

#como ya hemos traido html, sacamos una tabla de eso

def tabla_html_to_lista_tuplas(tabla_html): #el argumento es una tabla html
    lista_tuplas_oscars = []
    #print(tabla_html)
    filas = tabla_oscar.find_all("tr")
    for fila in filas:
        celdas = fila.find_all("td")
        lista_fila = []
        for celda in celdas:
            lista_fila.append(celda.text.strip())
        lista_tuplas_oscars.append(lista_fila)
    return lista_tuplas_oscars
    #nos la da sin limpiar

lista_tuplas_oscars = tabla_html_to_lista_tuplas(tabla_oscar)

#Ahora necesitamos que nos de la lista de tuplas "limpia"

def limpiar_lista_tuplas(lista_tuplas_oscars):
    lista_tuplas_limpia = []
    for fila in lista_tuplas_oscars:
        # quitamos la fila de cabecera que "está vacía", es decir, con etiqueta "th" no con "td" como las otras
        if fila != []:
            # llamamos a la función que hemos creado abajo para limpiar las filas
            fila_limpia = limpiar_fila(fila)
            lista_tuplas_limpia.append(fila_limpia)
    return lista_tuplas_limpia


# Ahora queremos limpiar cada fila. Le pasamos una fila sucia (tupla sucia) y 
# devuelve una fila limpia con el formato que nos piden:
# [(1990, 'Driving Miss Daisy', 'O. Stone', "D. Day-Lewis",' J. Tandy' ),  ...]

def limpiar_fila(fila_sucia):
    ceremonia = limpiar_celda_ceremonia(fila_sucia[1])
    pelicula = limpiar_celda_película(fila_sucia[2])
    director = limpiar_celda_nombres(fila_sucia[3])
    actor = limpiar_celda_nombres(fila_sucia[4])
    actriz = limpiar_celda_nombres (fila_sucia[5])
    fila_limpia = (ceremonia,pelicula,director,actor,actriz) # nos devuelve una tupla de todo lo anterior
    return fila_limpia 

def limpiar_celda_ceremonia(celda):
    celda_limpia = int(celda[-4:]) #convertimos a número el string que hemos separado y cogemos el 
                                             #ultimo string
    return celda_limpia

# la celda de Mejor Película ya me la da limpia la tabla original de html
# en caso de que haga falta cambiarla la dejamos hecha la función
def limpiar_celda_película(celda):
    return celda     

def limpiar_celda_nombres(celda):  # hay que limpiar el nombre de la pelicula
     celda_limpia = celda.split("(")[0] # que cuando se tope con "(" nos lo divida y nos quedamos con lo primero
     return celda_limpia


# Ahora hacemos la función de filtrar por año (1990-2023)
def filtrar_año(lista_tuplas_oscars,rango):
    lista_tuplas_oscars_filtrada = []
    for fila in lista_tuplas_oscars:
        if fila[0] in rango:
            lista_tuplas_oscars_filtrada.append(fila)

    return  lista_tuplas_oscars_filtrada

fila_limpia = limpiar_fila(lista_tuplas_oscars[80])


lista_tuplas_final = filtrar_año(limpiar_lista_tuplas(lista_tuplas_oscars),range(1990,2024))


**Fase 6: Inserción de Datos en la Base de Datos**

Primero realizaremos la conexión con MySQL y después haremos la inserción de los datos obtenidos en el punto anterior.

In [ ]:
def insertar_oscars(lista):
    """
    Esta función toma una lista de tuplas y la inserta en una tabla de la bbdd
    
    Args:
    - lista: listade tuplas donde cada tupla incluye los datos de los oscars de un año (fecha_ceremonia, nombre_pelicula, direccion, nombre_actor, nombre_actriz)

    Returns:
    - devuelve el número de registros insertados

    """
    contrasena = os.environ.get('contraseña')
    
    try:

        cnx = mysql.connector.connect(user='root', 
                                      password=contrasena,
                                      host='127.0.0.1',
                                      database='bho_peliculas_3')
        # en caso de que no lo consigas por que hay algún error entonces ...
    except mysql.connector.Error as err:

    # si es un error con la contraseña devuelveme un mensaje de acceso denegado ya que tenemos problemas con la contraseña
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        
        # si el error no tiene que ver con la contraseña, puede ser porque la base de datos no exista, devuelveme un mensaje de que la base de datos no existe
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        
        # si no es por ninguno de los errores anteriores, printeame cual es el error que estoy teniendo en mi conexión
        else:
            print(err)

    mycursor = cnx.cursor()
    sql = "INSERT INTO oscars (fecha_ceremonia, nombre_pelicula, direccion, nombre_actor, nombre_actriz) VALUES (%s, %s, %s, %s, %s)" 

    try: 
        mycursor.executemany(sql, lista)
        cnx.commit()

        print(mycursor.rowcount, "registro insertado/s.")

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

    cnx.close()    

    return mycursor.rowcount    

In [ ]:
insertar_oscars(lista_tuplas_final)